In [26]:
val spark = SparkSession
      .builder
      .master("local[1]")
      .getOrCreate

spark = org.apache.spark.sql.SparkSession@38a16f45


lastException: Throwable = null


org.apache.spark.sql.SparkSession@38a16f45

In [94]:
import java.net.URL
import java.net.URLDecoder.decode
import org.apache.spark.sql.functions._
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions.regexp_replace
import scala.util.Try
import org.apache.spark.sql.types.{IntegerType, LongType, StructType, DataType, StringType}
import java.net.URLDecoder._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import scala.util.Try
import java.net.URL
import java.net.URLDecoder.decode
import scala.util.Try
import org.apache.spark.sql.functions.{udf, col}
 import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{CountVectorizer, StringIndexer, IndexToString}
import org.apache.spark.ml.{Pipeline, PipelineModel}

lastException: Throwable = null


### TRAIN

In [69]:
    val trainPath = "/labs/slaba04/gender_age_dataset.txt"
    val modelPath = "/user/sergey.polyakhov/laba04/model/"
    val maxIter = "10"
    val regParam = "0.001" 

trainPath = /labs/slaba04/gender_age_dataset.txt
modelPath = /user/sergey.polyakhov/laba04/model/
maxIter = 10
regParam = 0.001


lastException: Throwable = null


0.001

In [70]:
 val exprGetDomain = "regexp_replace(parse_url(visits.url, 'HOST'), '^www.', '')"

exprGetDomain = regexp_replace(parse_url(visits.url, 'HOST'), '^www.', '')


regexp_replace(parse_url(visits.url, 'HOST'), '^www.', '')

In [71]:
  val data =spark.read
    .format("csv")
    .option("header", "true")
    .option("sep", "\t") 
    .option("mode", "failfast") 
    .load(trainPath)
    .withColumn("gender_age", concat('gender, lit("_"), 'age))
    //.select(col("uid"),col("gender"), col("age"), explode(col("visits")).alias("visits"))

data = [gender: string, age: string ... 3 more fields]


[gender: string, age: string ... 3 more fields]

In [72]:
val schema = spark.read.json(data.select("user_json").as[String]).schema

schema = StructType(StructField(visits,ArrayType(StructType(StructField(timestamp,LongType,true), StructField(url,StringType,true)),true),true))


StructType(StructField(visits,ArrayType(StructType(StructField(timestamp,LongType,true), StructField(url,StringType,true)),true),true))

In [73]:
val logs = data
.select($"uid", $"gender_age", from_json($"user_json", schema).as("s"))
.select("uid", "gender_age", "s.*")
.select(col("uid"),col("gender_age"), explode(col("visits")).alias("visits"))
.withColumn("domain", expr(exprGetDomain))
.groupBy(col("uid"), col("gender_age"))
.agg(collect_list("domain").alias("domains")) 

logs = [uid: string, gender_age: string ... 1 more field]


[uid: string, gender_age: string ... 1 more field]

In [74]:
logs.printSchema

root
 |-- uid: string (nullable = true)
 |-- gender_age: string (nullable = true)
 |-- domains: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [78]:
import org.apache.spark.sql.functions._

In [79]:
val l_test=
logs
.select(col("uid"),col("gender_age"))
.withColumn("gender", substring_index(col("gender_age"), "_",1))
.withColumn("age", substring_index(col("gender_age"), "_",-1))
.cache()



l_test = [uid: string, gender_age: string ... 2 more fields]


[uid: string, gender_age: string ... 2 more fields]

In [80]:
l_test.show(2, false)

+------------------------------------+----------+------+-----+
|uid                                 |gender_age|gender|age  |
+------------------------------------+----------+------+-----+
|d52ab244-0a9c-434f-9dd0-34594b2a5993|M_25-34   |M     |25-34|
|d60b267c-dab6-46a5-81f8-463d7deb6a60|F_25-34   |F     |25-34|
+------------------------------------+----------+------+-----+
only showing top 2 rows



In [11]:
val cv = new CountVectorizer()
      .setInputCol("domains")
      .setOutputCol("features")

cv = cntVec_0a683efb0b61


cntVec_0a683efb0b61

In [12]:
 val indexer = new StringIndexer()
      .setInputCol("gender_age")
      .setOutputCol("label")
      .fit(logs)

indexer = strIdx_9ebce3aaed00


strIdx_9ebce3aaed00

In [13]:
 val lr = new LogisticRegression()
      .setMaxIter(maxIter.toInt)
      .setRegParam(regParam.toDouble)

lr = logreg_6ff5683c007c


logreg_6ff5683c007c

In [14]:
val converter = new IndexToString()
      .setInputCol("prediction")
      .setOutputCol("label_string")
      .setLabels(indexer.labels)

converter = idxToStr_92c0ef42c7e5


idxToStr_92c0ef42c7e5

In [15]:
val pipeline = new Pipeline()
      .setStages(Array(cv, indexer, lr, converter)) 

pipeline = pipeline_e28e4ac5f75b


pipeline_e28e4ac5f75b

In [16]:
 val model = pipeline.fit(logs)
    model.write.overwrite.save(modelPath) 

model = pipeline_e28e4ac5f75b


pipeline_e28e4ac5f75b

### TEST

In [27]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.ml.{Pipeline, PipelineModel}

In [28]:
val modelPath = "/user/sergey.polyakhov/laba04/model/"
val kafkaInputTopic = "input_sergey.polyakhov"
val kafkaOutputTopic = "sergey.polyakhov"
val kafkaServer = "spark-de-master-1.newprolab.com:6667"

modelPath = /user/sergey.polyakhov/laba04/model/
kafkaInputTopic = input_sergey.polyakhov
kafkaOutputTopic = sergey.polyakhov
kafkaServer = spark-de-master-1.newprolab.com:6667


spark-de-master-1.newprolab.com:6667

In [29]:
val model = PipelineModel.load(modelPath)

model = pipeline_e28e4ac5f75b


pipeline_e28e4ac5f75b

In [30]:
  val json_schema1 = StructType(Seq(
      StructField("uid", StringType, nullable = true),
      StructField("visits", StringType, nullable = true) ))


schema = StructType(StructField(uid,StringType,true), StructField(visits,ArrayType(StructType(StructField(url,StringType,true), StructField(timestamp,StringType,true)),true),true))


StructType(StructField(uid,StringType,true), StructField(visits,ArrayType(StructType(StructField(url,StringType,true), StructField(timestamp,StringType,true)),true),true))

In [ ]:
val json_schema2 = ArrayType(StructType(Seq(StructField("url", StringType), StructField("timestamp", StringType))))

In [31]:
val exprGetDomain = "regexp_replace(parse_url(visits, 'HOST'), '^www.', '')"

exprGetDomain = regexp_replace(parse_url(visits.url, 'HOST'), '^www.', '')


regexp_replace(parse_url(visits.url, 'HOST'), '^www.', '')

In [17]:
val logs = spark
      .read
      .format("kafka")
      .option("kafka.bootstrap.servers", kafkaServer)
      .option("subscribe", kafkaInputTopic)
      .option("startingOffsets", "earliest")
      .option("checkpointLocation", s"chk/step2_lab04")
      .load


logs = [key: binary, value: binary ... 5 more fields]


lastException: Throwable = null


[key: binary, value: binary ... 5 more fields]

In [47]:
 val preparedLogs = logs
          .select(col("timestamp"), from_json(col("value").cast("string"), json_schema1).alias("valueParsed"))
            .select(col("valueParsed.*"))
            .select($"uid", from_json($"visits", json_schema2).as("s"))
            .select("uid", "s.url")
            .select(col("uid"), explode(col("url")).alias("visits"))
            .withColumn("domain", expr(exprGetDomain))
            .groupBy(col("uid"))
            .agg(collect_list("domain").alias("domains")) 
            .cache()

preparedLogs = [uid: string, domains: array<string>]


[uid: string, domains: array<string>]

In [48]:
val result = model.transform(preparedLogs).cache()

0

In [ ]:
result
          .select( col("uid"),col("label_string").alias("gender_age"))
          .withColumn("gender", substring_index(col("gender_age"), "_",1))
          .withColumn("age", substring_index(col("gender_age"), "_",-1))
          .select(col("uid"),col("gender"), col("age"))
           .toJSON
           .write
          //  .trigger(Trigger.ProcessingTime("5 seconds"))
          .format("kafka")
          .option("kafka.bootstrap.servers", kafkaServer)
          .option("topic", kafkaOutputTopic)
           .save()

In [ ]:
spark.stop